SCRAPING ITEM'S DATA FROM SHOPEE WEBSITE

In [1]:
import requests
import json
import sqlite3 

In [2]:
def get_shop(category_id): 

    url = "https://shopee.vn/api/v4/official_shop/get_shops?category_id=" + f'{category_id}'
    headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.0.0'
    }
    response = requests.get(url, headers= headers)

    data = response.json()
    
    for i in data['data']['official_shops']:
        shop_id = i['shopid']
        row = i['shopid'], i['shop_name']
        cursor.execute('INSERT OR REPLACE INTO shop VALUES(?,?)',row)

In [3]:
def get_item(number,shop_id): 

    url = 'https://shopee.vn/api/v4/shop/rcmd_items?bundle=shop_page_category_tab_main&limit=100&offset=' + f'{number}' + '&shop_id=' + f'{shop_id}' + '&sort_type=1&upstream='

    response = requests.get(url)

    data = response.json()
    data = data['data']['items']

    
    for i in data:
        item_id = i['itemid']
        row = i['itemid'], i['shopid'], i['name'], i['stock'], i['historical_sold'], i['liked_count'], i['cmt_count'], i['item_status'],\
            i['price']/100, i['price_min']/100, i['price_max']/100, i['raw_discount'], i['item_rating']['rating_count'][1], i['item_rating']['rating_count'][2], i['item_rating']['rating_count'][3],\
            i['item_rating']['rating_count'][4], i['item_rating']['rating_count'][5],
        cursor.execute('INSERT OR REPLACE INTO item VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',row)


In [8]:
conn = sqlite3.connect('shopee.db')
cursor = conn.cursor()

get_shop(11035567) #Thời trang nam
conn.commit()

cursor.execute('SELECT id FROM shop')
shop_id_list = [row[0] for row in cursor.fetchall()]

for x in shop_id_list:
    total_item = requests.get('https://shopee.vn/api/v4/shop/rcmd_items?bundle=shop_page_category_tab_main&limit=1&offset=0&shop_id=' + f'{x}' + '&sort_type=1&upstream=').json()['data']['total']
    for i in range(0,total_item,100):
        get_item(i,x)

conn.commit()
conn.close()